# 1. Samsung

In [1]:
import pandas as pd

def get_support_timeline(device, dataset):
    # Copy the dataset to avoid modifying the original
    df = dataset.copy()
    
    df['date'] = pd.to_datetime(df['date'])
    
    # Filter data for the specific device and sort by date
    df_device = df[df['device'] == device].sort_values(by='date', ascending=True)
    
    # Initialize variables for start and end dates for each support period
    try:
        monthly_start = df_device[df_device.support_type == 'monthly'].iloc[0]['date']
        monthly_end = df_device[df_device.support_type == 'monthly'].iloc[-1]['date']
    except IndexError:
        monthly_start = None
        monthly_end = None
    
    try: 
        quarterly_start = df_device[df_device.support_type == 'quarterly'].iloc[0]['date']
        quarterly_end = df_device[df_device.support_type == 'quarterly'].iloc[-1]['date']
    except IndexError:
        quarterly_start = None
        quarterly_end = None
    
    try:
        biannual_start = df_device[df_device.support_type == 'biannual'].iloc[0]['date']
        biannual_end = df_device[df_device.support_type == 'biannual'].iloc[-1]['date']
    except IndexError:
        biannual_start = None
        biannual_end = None

    # Store the results in a list
    result = [monthly_start, monthly_end, quarterly_start, quarterly_end, biannual_start, biannual_end]
    
    return result


In [2]:
import pandas as pd

df_final_samsung = pd.read_csv('../2024_Data/samsung/combined_support_lists.csv')

# Filter the date to include on data by May 2024
df_final_samsung['date'] = pd.to_datetime(df_final_samsung['date'])
df_final_samsung = df_final_samsung[df_final_samsung['date'] < '2024-06-01']

print('Unique devices:', len(set(df_final_samsung['device'])))
counts_by_period = df_final_samsung.groupby("support_type")['device'].nunique()
print(counts_by_period)


Unique devices: 340
support_type
biannual     125
monthly      122
quarterly    294
Name: device, dtype: int64


In [3]:
df_timelines_samsung = pd.DataFrame()
for device in df_final_samsung['device'].unique():
    
    support_timeline = get_support_timeline(device, df_final_samsung)
    
    df_new = pd.DataFrame({"device": device, 
                           "monthly_start": support_timeline[0],
                           "monthly_end": support_timeline[1],
                           "quarterly_start": support_timeline[2],
                           "quarterly_end": support_timeline[3],
                           "biannual_start": support_timeline[4],
                           "biannual_end": support_timeline[5]},
                          index=[0])
 
    # Exclude empty or all-NA columns
    df_new = df_new.dropna(axis=1, how='all')
    
    df_timelines_samsung = pd.concat([df_timelines_samsung, df_new], ignore_index=True)

df_timelines_samsung.to_csv('../2024_Data/samsung/timelines_samsung_v1.csv', index=False)

df_timelines_samsung.head()


,device,monthly_start,monthly_end,quarterly_start,quarterly_end,biannual_start,biannual_end
0,Galaxy Z Fold2,2020-09-14,2023-10-03,2020-09-14,2024-04-04,NaT,NaT
1,Galaxy Z Fold2 5G,2020-09-14,2024-05-27,2020-09-14,2024-05-27,NaT,NaT
2,Galaxy Z Fold3 5G,2021-09-07,2024-05-27,NaT,NaT,NaT,NaT
3,Galaxy Z Fold4,2022-09-07,2024-05-27,NaT,NaT,NaT,NaT
4,Galaxy Z Flip,2020-03-03,2023-04-04,2020-03-03,2024-03-31,NaT,NaT


## Samsung Support List Stats

In [4]:
import pandas as pd

first_snapshot = df_final_samsung['date'].min()
last_snapshot = df_final_samsung['date'].max()
all_snapshots = len(df_final_samsung['date'].unique())
avg_snapshot_interval = (last_snapshot - first_snapshot).days / all_snapshots
unique_devices_total = len(df_timelines_samsung.index)
unique_devices_last_list = len(df_final_samsung[df_final_samsung['date'] == last_snapshot])

print('First snapshot: {}'.format(first_snapshot))
print('Last snapshot: {}'.format(last_snapshot))
print('All snapshots: {}'.format(all_snapshots))
print('Avg snapshot interval (days): {}'.format(avg_snapshot_interval))
print('Unique Devices (Total): {}'.format(unique_devices_total))
print('Unique Devices (Last list): {}'.format(unique_devices_last_list))


First snapshot: 2017-10-03 00:00:00
Last snapshot: 2024-05-27 00:00:00
All snapshots: 524
Avg snapshot interval (days): 4.633587786259542
Unique Devices (Total): 340
Unique Devices (Last list): 149


### Filtering the incomplete periods

Removing first and last snaphot from the beginning and end of the period because:
- as devices in the last snapshot are likely to continue to receive security updates, their end date cannot be marked yet.
- similarly, when the support lists started to be published, the devices given in the first published support list were probably getting the support before that date, and the start of their support period cannot be marked as we have no earlier data than the first snapshot.


In [5]:
import pandas as pd

df = pd.read_csv('../2024_Data/samsung/timelines_samsung_v1.csv')

# Convert date columns to datetime format
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')

# Define the first and last dates to be removed
date_first_list = pd.Timestamp('2017-10-03 00:00:00')
date_last_list = pd.Timestamp('2024-05-27 00:00:00')

# Iterate over columns except the first (device) column
for column in df.columns[1:]:
    df[column] = df[column].apply(lambda x: pd.NaT if x in [date_first_list, date_last_list] else x)

df.to_csv('../2024_Data/samsung/timelines_samsung_v2.csv', index=False)
df.head()


,device,monthly_start,monthly_end,quarterly_start,quarterly_end,biannual_start,biannual_end
0,Galaxy Z Fold2,2020-09-14,2023-10-03,2020-09-14,2024-04-04,NaT,NaT
1,Galaxy Z Fold2 5G,2020-09-14,NaT,2020-09-14,NaT,NaT,NaT
2,Galaxy Z Fold3 5G,2021-09-07,NaT,NaT,NaT,NaT,NaT
3,Galaxy Z Fold4,2022-09-07,NaT,NaT,NaT,NaT,NaT
4,Galaxy Z Flip,2020-03-03,2023-04-04,2020-03-03,2024-03-31,NaT,NaT


## Calculating average durations in the support lists

- Also filter out if only one time appeared in the list since duration can not be calculated with a single date

In [6]:
df_timelines = pd.read_csv('../2024_Data/samsung/timelines_samsung_v2.csv')
    
df_timelines[['monthly_start','monthly_end','quarterly_start','quarterly_end','biannual_start','biannual_end']] = df_timelines[['monthly_start','monthly_end','quarterly_start','quarterly_end','biannual_start','biannual_end']].apply(pd.to_datetime)
df_timelines['monthly_duration'] = df_timelines['monthly_end']-df_timelines['monthly_start']
df_timelines['quarterly_duration'] = df_timelines['quarterly_end']-df_timelines['quarterly_start']
df_timelines['biannual_duration'] = df_timelines['biannual_end']-df_timelines['biannual_start']

df_timelines.to_csv('../2024_Data/samsung/timelines_samsung_v3.csv',index=False)

cond1 = ~(df_timelines['monthly_duration'].dt.days==0) #filter out if only one time appeared in the list
cond2 = ~df_timelines['monthly_duration'].isnull() #filter out as no monthly duration can be calculated

df_timelines_monthly = df_timelines[cond1&cond2].reset_index(drop=True)

cond3 = ~(df_timelines['quarterly_duration'].dt.days==0) #filter out if only one time appeared in the list
cond4 = ~df_timelines['quarterly_duration'].isnull() #filter out as no monthly duration can be calculated

df_timelines_quarterly = df_timelines[cond3&cond4].reset_index(drop=True)

cond5 = ~(df_timelines['biannual_duration'].dt.days==0) #filter out if only one time appeared in the list
cond6 = ~df_timelines['biannual_duration'].isnull() #filter out as no monthly duration can be calculated

df_timelines_biannual = df_timelines[cond5&cond6].reset_index(drop=True)

print("Avg monthly support duration: {}".format(df_timelines_monthly['monthly_duration'].mean(skipna=True)))
print('Monthly completed periods for: {} devices'.format(len(df_timelines_monthly['device'].unique())))

print("Avg quarterly support duration: {}".format(df_timelines_quarterly['quarterly_duration'].mean(skipna=True)))
print('Quarterly completed periods for: {} devices'.format(len(df_timelines_quarterly['device'].unique())))

print("Avg binannually support duration: {}".format(df_timelines_biannual['biannual_duration'].mean(skipna=True)))
print('Biannual completed periods for: {} devices'.format(len(df_timelines_biannual['device'].unique())))

Avg monthly support duration: 850 days 16:19:12
Monthly completed periods for: 50 devices
Avg quarterly support duration: 689 days 18:54:18.100558656
Quarterly completed periods for: 179 devices
Avg binannually support duration: 634 days 00:16:56.470588232
Biannual completed periods for: 85 devices


## Tablets-only

- Noticed that no tablets is given in monthly support list

In [7]:
df_timelines_tablets = df_timelines[df_timelines['device'].str.contains('Tab')]

df_timelines_tablets.head()

,device,monthly_start,monthly_end,quarterly_start,quarterly_end,biannual_start,biannual_end,monthly_duration,quarterly_duration,biannual_duration
86,Galaxy Tab A7 Lite,NaT,NaT,2021-06-08,2023-06-05,2023-06-07,NaT,NaT,727 days,NaT
87,Galaxy Tab A8,NaT,NaT,2022-01-12,2024-01-31,2024-02-06,NaT,NaT,749 days,NaT
88,Galaxy Tab Active Pro,NaT,NaT,2019-10-08,2023-11-06,NaT,NaT,NaT,1490 days,NaT
89,Galaxy Tab Active3,NaT,NaT,2020-10-12,NaT,NaT,NaT,NaT,NaT,NaT
90,Galaxy Tab Active4 Pro,NaT,NaT,2022-09-07,NaT,NaT,NaT,NaT,NaT,NaT


## Watch-only

In [8]:
df_timelines_watch = df_timelines[df_timelines['device'].str.contains('Watch')]

df_timelines_watch.head()

,device,monthly_start,monthly_end,quarterly_start,quarterly_end,biannual_start,biannual_end,monthly_duration,quarterly_duration,biannual_duration
155,Galaxy Watch4,NaT,NaT,2023-01-04,NaT,NaT,NaT,NaT,NaT,NaT
156,Galaxy Watch4 Classic,NaT,NaT,2023-01-04,NaT,NaT,NaT,NaT,NaT,NaT
157,Galaxy Watch5,NaT,NaT,2023-01-04,NaT,NaT,NaT,NaT,NaT,NaT
158,Galaxy Watch5 Pro,NaT,NaT,2023-01-04,NaT,NaT,NaT,NaT,NaT,NaT
202,Galaxy Watch6,NaT,NaT,2023-08-08,NaT,NaT,NaT,NaT,NaT,NaT


# 2. Xiaomi

## Xiaomi Support Lists

In [9]:
import glob 
from datetime import datetime, date
import os


def excel_to_dataframe(excel_path):
    df1 = pd.read_excel(excel_path, sheet_name = 'Monthly', usecols = 'A', names=['phone_model'],header=None, engine='openpyxl')
    df1['support'] = 'Monthly'
    df1['date'] = datetime.strptime(os.path.splitext(os.path.basename(excel_path))[0],'%Y%m%d%H%M%S')

    df2 = pd.read_excel(excel_path, sheet_name = 'Quarterly', usecols = 'A', names=['phone_model'],header=None, engine='openpyxl')
    df2['support'] = 'Quarterly'
    df2['date'] = datetime.strptime(os.path.splitext(os.path.basename(excel_path))[0],'%Y%m%d%H%M%S')

    try:
        df3 = pd.read_excel(excel_path, sheet_name = 'Other', usecols = 'A', names=['phone_model'],header=None, engine='openpyxl')
        df3['support'] = 'Biannually'
        df3['date'] = datetime.strptime(os.path.splitext(os.path.basename(excel_path))[0],'%Y%m%d%H%M%S')
    except ValueError:
        try: 
            df3 = pd.read_excel(excel_path, sheet_name = 'Biannual', usecols = 'A', names=['phone_model'],header=None, engine='openpyxl')
            df3['support'] = 'Biannually'
            df3['date'] = datetime.strptime(os.path.splitext(os.path.basename(excel_path))[0],'%Y%m%d%H%M%S')
        except ValueError:
            df3 = pd.DataFrame([])
            
    try:
        df4 = pd.read_excel(excel_path, sheet_name = 'Wearable', usecols = 'A', names=['phone_model'],header=None, engine='openpyxl')
        df4['support'] = 'Wearable'
        df4['date'] = datetime.strptime(os.path.splitext(os.path.basename(excel_path))[0],'%Y%m%d%H%M%S')
    except ValueError:
        df4 = pd.DataFrame([])

    df = pd.concat([df1,df2,df3,df4])
    return df


def get_support_timeline(phone,dataset):

    df = dataset
    
    #print('Getting support timeline for {}!'.format(phone))
    
    df_phone = df[df.phone_model == phone].sort_values(by='date')
    
    try:
        monthly_start = df_phone[df_phone.support == 'Monthly'].iloc[0]['date']
        monthly_end = df_phone[df_phone.support == 'Monthly'].iloc[-1]['date']
    except IndexError:
        monthly_start = None
        monthly_end = None
        
    try: 
        quarterly_start = df_phone[df_phone.support == 'Quarterly'].iloc[0]['date']
        quarterly_end = df_phone[df_phone.support == 'Quarterly'].iloc[-1]['date']
    except IndexError:
        quarterly_start = None
        quarterly_end = None
        
    try:
        biannually_start = df_phone[df_phone.support == 'Biannually'].iloc[0]['date']
        biannually_end = df_phone[df_phone.support == 'Biannually'].iloc[-1]['date']
    except IndexError:
        biannually_start = None
        biannually_end = None
    
    try:
        wearable_start = df_phone[df_phone.support == 'Wearable'].iloc[0]['date']
        wearable_end = df_phone[df_phone.support == 'Wearable'].iloc[-1]['date']
    except IndexError:
        wearable_start = None
        wearable_end = None
    result = [monthly_start,monthly_end,quarterly_start,quarterly_end,biannually_start,biannually_end,wearable_start,wearable_end]
    
    return result



dir_path = '../2024_Data/xiaomi/xiaomi_support_lists/'

print('Getting lists from {}'.format(dir_path))

excel_files = glob.glob(dir_path + '*.xlsx')
df_final_xiaomi = pd.DataFrame([])

for file in excel_files[:]:
    print('Processing file:', file)
    df_new = excel_to_dataframe(file)
    df_final_xiaomi = pd.concat([df_final_xiaomi,df_new],ignore_index=True)

print('Converting excel files to DataFrame is done!')

df_timelines_xiaomi = pd.DataFrame([])
for phone in df_final_xiaomi['phone_model'].unique():
    
    support_timeline = get_support_timeline(phone,df_final_xiaomi)
    
    df_new = pd.DataFrame({"phone": phone, 
                           "monthly_start": support_timeline[0],
                           "monthly_end": support_timeline[1],
                           "quarterly_start": support_timeline[2],
                           "quarterly_end": support_timeline[3],
                           "biannually_start": support_timeline[4],
                           "biannually_end": support_timeline[5],
                           "wearable_start": support_timeline[6],
                           "wearable_end": support_timeline[7]},
                          index=[0])
 
    
    df_timelines_xiaomi = pd.concat([df_timelines_xiaomi,df_new],ignore_index=True)
    
df_timelines_xiaomi.to_csv('../2024_Data/xiaomi/timelines_xiaomi.csv',index=False)

df_timelines_xiaomi.head()


Getting lists from ../2024_Data/xiaomi/xiaomi_support_lists/
Processing file: ../2024_Data/xiaomi/xiaomi_support_lists/20210311195400.xlsx
Processing file: ../2024_Data/xiaomi/xiaomi_support_lists/20210127174516.xlsx
Processing file: ../2024_Data/xiaomi/xiaomi_support_lists/20210217113901.xlsx
Processing file: ../2024_Data/xiaomi/xiaomi_support_lists/20210506224538.xlsx
Processing file: ../2024_Data/xiaomi/xiaomi_support_lists/20210316092427.xlsx
Processing file: ../2024_Data/xiaomi/xiaomi_support_lists/20211012102400.xlsx
Processing file: ../2024_Data/xiaomi/xiaomi_support_lists/20221031175222.xlsx
Processing file: ../2024_Data/xiaomi/xiaomi_support_lists/20211208120946.xlsx
Converting excel files to DataFrame is done!


/var/folders/64/c68452391mv_849dpqtr2rch0000gn/T/ipykernel_60299/1949711739.py:110: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_timelines_xiaomi = pd.concat([df_timelines_xiaomi,df_new],ignore_index=True)


,phone,monthly_start,monthly_end,quarterly_start,quarterly_end,biannually_start,biannually_end,wearable_start,wearable_end
0,Mi A3,2021-01-27 17:45:16,2022-10-31 17:52:22,2021-01-27 17:45:16,2022-10-31 17:52:22,None,None,None,None
1,Mi A2,2021-01-27 17:45:16,2022-10-31 17:52:22,NaT,NaT,None,None,None,None
2,Mi A2 Lite,2021-01-27 17:45:16,2022-10-31 17:52:22,NaT,NaT,None,None,None,None
3,Mi 10T Pro,NaT,NaT,2021-01-27 17:45:16,2022-10-31 17:52:22,None,None,None,None
4,Mi 10 T,NaT,NaT,2021-01-27 17:45:16,2022-10-31 17:52:22,None,None,None,None


## Xiaomi EOL lists

In [10]:
folder_path = '../2024_Data/xiaomi/xiaomi_eol_devices/'

dfs = []

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        timestamp = filename.split('.')[0]
        date = datetime.strptime(timestamp, '%Y%m%d%H%M%S').date() 
        excel_data = pd.read_excel(os.path.join(folder_path, filename))

        if not excel_data.empty:
            source_url = excel_data['Source URL'].iloc[0]
            my_list = list(set(excel_data['Mi'].to_list()+excel_data['Redmi'].to_list()+excel_data['POCO'].to_list()))

            cleanedList = [x for x in my_list if str(x) != 'nan']

            # Create a temporary DataFrame from cleanedList and append it to dfs list
            temp_df = pd.DataFrame({
                'Date': [date]*len(cleanedList), 
                'Brand': cleanedList, 
                'Source URL': [source_url]*len(cleanedList)
            })
            dfs.append(temp_df)

# Concatenate all DataFrames in the dfs list
df_xiaomi_eol = pd.concat(dfs, ignore_index=True)
df_xiaomi_eol = df_xiaomi_eol.sort_values('Date')
df_xiaomi_eol.head()


,Date,Brand,Source URL
2787,2021-10-12,MI Note,https://web.archive.org/web/20211012105859/htt...
2777,2021-10-12,Redmi Note 1S,https://web.archive.org/web/20211012105859/htt...
2778,2021-10-12,MI Pad,https://web.archive.org/web/20211012105859/htt...
2779,2021-10-12,MI 1,https://web.archive.org/web/20211012105859/htt...
2781,2021-10-12,MI MAX 2,https://web.archive.org/web/20211012105859/htt...


In [11]:
# Convert the string columns to datetime format
df_timelines_xiaomi['monthly_end'] = pd.to_datetime(df_timelines_xiaomi['monthly_end'])
df_timelines_xiaomi['quarterly_end'] = pd.to_datetime(df_timelines_xiaomi['quarterly_end'])
df_timelines_xiaomi['biannually_end'] = pd.to_datetime(df_timelines_xiaomi['biannually_end'])

# Get the maximum date from the three columns
max_date = df_timelines_xiaomi[['monthly_end', 'quarterly_end', 'biannually_end']].max().max()

# Filter the rows where the date in any of the three columns matches the max_date
matching_rows = df_timelines_xiaomi[
    (df_timelines_xiaomi['monthly_end'] == max_date) |
    (df_timelines_xiaomi['quarterly_end'] == max_date) |
    (df_timelines_xiaomi['biannually_end'] == max_date)
]

print('Unique devices (last support list): {}'.format(len(matching_rows)))


Unique devices (last support list): 52


In [12]:
print('Unique devices (all support lists): {}'.format(len(df_timelines_xiaomi['phone'].unique())))


Unique devices (all support lists): 52


In [13]:
latest_date = df_xiaomi_eol['Date'].max()
rows_matching_latest_date = df_xiaomi_eol[df_xiaomi_eol['Date'] == latest_date]

print('Unique devices (last EOL list): {}'.format(len(rows_matching_latest_date)))


Unique devices (last EOL list): 174


In [14]:
print('Unique devices (all EOL lists): {}'.format(len(df_xiaomi_eol['Brand'].unique())))

Unique devices (all EOL lists): 217


In [15]:
appearance_df = pd.DataFrame({
    'First Appearance': df_xiaomi_eol.groupby('Brand')['Date'].min(),
    'Last Appearance': df_xiaomi_eol.groupby('Brand')['Date'].max()
})

# Sort appearance_df by 'First Appearance'
appearance_df = appearance_df.sort_values('First Appearance')

appearance_df.head()

,First Appearance,Last Appearance
Brand,,
MI 1,2021-10-12,2022-08-24
MIX 2,2021-10-12,2024-06-23
MIX 2S,2021-10-12,2023-03-06
MIX 3,2021-10-12,2023-03-06
Redmi Y2,2021-10-12,2024-06-23


# 3. Oppo

In [16]:
def excel_to_dataframe(excel_path):
    df1 = pd.read_excel(excel_path, sheet_name='monthly', usecols='A', names=['phone_model'], header=None, engine='openpyxl')
    df1['support'] = 'monthly'
    df1['date'] = datetime.strptime(os.path.splitext(os.path.basename(excel_path))[0], '%Y%m%d%H%M%S')

    df2 = pd.read_excel(excel_path, sheet_name='quarterly', usecols='A', names=['phone_model'], header=None, engine='openpyxl')
    df2['support'] = 'quarterly'
    df2['date'] = datetime.strptime(os.path.splitext(os.path.basename(excel_path))[0], '%Y%m%d%H%M%S')

    df3 = pd.read_excel(excel_path, sheet_name='others', usecols='A', names=['phone_model'], header=None, engine='openpyxl')
    df3['support'] = 'others'
    df3['date'] = datetime.strptime(os.path.splitext(os.path.basename(excel_path))[0], '%Y%m%d%H%M%S')

    df = pd.concat([df1, df2, df3])
    return df



def get_support_timeline(phone,dataset):
    df = dataset
    #print('Getting support timeline for {}!'.format(phone))
    df_phone = df[df.phone_model == phone].sort_values(by='date')
    try:
        monthly_start = df_phone[df_phone.support == 'monthly'].iloc[0]['date']
        monthly_end = df_phone[df_phone.support == 'monthly'].iloc[-1]['date']
    except IndexError:
        monthly_start = None
        monthly_end = None
  
    try: 
        quarterly_start = df_phone[df_phone.support == 'quarterly'].iloc[0]['date']
        quarterly_end = df_phone[df_phone.support == 'quarterly'].iloc[-1]['date']
    except IndexError:
        quarterly_start = None
        quarterly_end = None
        
    try: 
        others_start = df_phone[df_phone.support == 'others'].iloc[0]['date']
        others_end = df_phone[df_phone.support == 'others'].iloc[-1]['date']
    except IndexError:
        others_start = None
        others_end = None
        
    result = [monthly_start,monthly_end,quarterly_start,quarterly_end,others_start,others_end]
    
    return result

In [17]:
import glob
from datetime import datetime, date
import os


dir_path = '../2024_Data/oppo/oppo_support_lists/'
print('Getting lists from {}'.format(dir_path))

# Use os.listdir() to get the list of files in the directory
files = os.listdir(dir_path)
df_final_oppo = pd.DataFrame()

# Iterate over the files
for file in files:
    if not file.startswith('~$') and file.endswith('.xlsx'):
        file_path = os.path.join(dir_path, file)
        print('Processing file:', file_path)
        df_new = excel_to_dataframe(file_path)
        df_final_oppo = pd.concat([df_final_oppo, df_new], ignore_index=True)

print('Converting excel files to DataFrame is done!')

df_final_oppo.head()


Getting lists from ../2024_Data/oppo/oppo_support_lists/
Processing file: ../2024_Data/oppo/oppo_support_lists/20230303162208.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/20220124115149.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/20240531043250.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/20210909091021.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/20220225225621.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/20210313062821.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/20230307192415.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/20210121171659.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/20211123130559.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/20210123012929.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/20220609072331.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/20220313142503.xlsx
Processing file: ../2024_Data/oppo/oppo_support_lists/2

,phone_model,support,date
0,Find X2（CPH2023、CPH1921）,monthly,2023-03-03 16:22:08
1,Find X2 Pro（CPH2025）,monthly,2023-03-03 16:22:08
2,Find X3（PEDM00）,monthly,2023-03-03 16:22:08
3,Find X3 Neo（CPH2207）,monthly,2023-03-03 16:22:08
4,Find X3 Pro（PEEM00、CPH2173、CPH2305）,monthly,2023-03-03 16:22:08


In [18]:
import pandas as pd

# Split the column values into two columns
df_final_oppo[['model_name', 'model_number']] = df_final_oppo['phone_model'].str.split('（', expand=True)
df_final_oppo['model_number'] = df_final_oppo['model_number'].str.replace('）', '')
df_final_oppo['model_name'] = df_final_oppo['model_name'].str.split('(', expand=True)[0]

# Save to CSV with explicit UTF-8 encoding
csv_file_path = '../2024_Data/oppo/oppo_support_lists.csv'
df_final_oppo.to_csv(csv_file_path, index=False, encoding="utf-8")

# Check the saved CSV file
df_saved = pd.read_csv(csv_file_path)
print(df_saved.head())


                           phone_model  support                 date  \
0             Find X2（CPH2023、CPH1921）  monthly  2023-03-03 16:22:08   
1                 Find X2 Pro（CPH2025）  monthly  2023-03-03 16:22:08   
2                      Find X3（PEDM00）  monthly  2023-03-03 16:22:08   
3                 Find X3 Neo（CPH2207）  monthly  2023-03-03 16:22:08   
4  Find X3 Pro（PEEM00、CPH2173、CPH2305）  monthly  2023-03-03 16:22:08   

    model_name            model_number  
0      Find X2         CPH2023、CPH1921  
1  Find X2 Pro                 CPH2025  
2      Find X3                  PEDM00  
3  Find X3 Neo                 CPH2207  
4  Find X3 Pro  PEEM00、CPH2173、CPH2305  


In [19]:
print('Unique devices:', len(set(df_saved['phone_model'])))
counts_by_period = df_saved.groupby("support")['phone_model'].nunique()
print(counts_by_period)


Unique devices: 329
support
monthly       21
others       117
quarterly    222
Name: phone_model, dtype: int64


In [20]:
import glob
from datetime import datetime, date
import os

df_timelines_oppo = pd.DataFrame([])
for phone in df_final_oppo['phone_model'].unique():
    
    support_timeline = get_support_timeline(phone, df_final_oppo)
    
    df_new = pd.DataFrame({"phone": phone, 
                           "monthly_start": support_timeline[0],
                           "monthly_end": support_timeline[1],
                           "quarterly_start": support_timeline[2],
                           "quarterly_end": support_timeline[3],
                           "others_start": support_timeline[4],
                           "others_end": support_timeline[5]},
                          index=[0])
 
    
    df_timelines_oppo = pd.concat([df_timelines_oppo,df_new],ignore_index=True)

df_timelines_oppo.head()

/var/folders/64/c68452391mv_849dpqtr2rch0000gn/T/ipykernel_60299/1035765439.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_timelines_oppo = pd.concat([df_timelines_oppo,df_new],ignore_index=True)
/var/folders/64/c68452391mv_849dpqtr2rch0000gn/T/ipykernel_60299/1035765439.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_timelines_oppo = pd.concat([df_timelines_oppo,df_new],ignore_index=True)
/var/folders/64/c68452391mv_849dpqtr2rch0000gn/T/ipykernel_60299/1035765439.py:20: FutureW

,phone,monthly_start,monthly_end,quarterly_start,quarterly_end,others_start,others_end
0,Find X2（CPH2023、CPH1921）,2022-10-11 18:51:30,2024-05-31 04:32:50,NaT,NaT,NaT,NaT
1,Find X2 Pro（CPH2025）,2022-10-11 18:51:30,2023-03-07 19:24:15,NaT,NaT,NaT,NaT
2,Find X3（PEDM00）,2022-10-11 18:51:30,2024-05-31 04:32:50,NaT,NaT,2021-01-21 17:16:59,2022-06-09 07:23:31
3,Find X3 Neo（CPH2207）,2022-10-11 18:51:30,2024-05-31 04:32:50,NaT,NaT,2021-01-21 17:16:59,2022-06-09 07:23:31
4,Find X3 Pro（PEEM00、CPH2173、CPH2305）,2022-10-11 18:51:30,2024-05-31 04:32:50,NaT,NaT,NaT,NaT


In [21]:
# split the column values into two columns
df_timelines_oppo[['model_name', 'model_number']] = df_timelines_oppo['phone'].str.split('（', expand=True)
df_timelines_oppo['model_number'] = df_timelines_oppo['model_number'].str.replace('）', '')
df_timelines_oppo['model_name_v2'] = df_timelines_oppo['model_name'].str.split('(', expand=True)[0]

df_timelines_oppo['monthly_duration'] = (df_timelines_oppo['monthly_end']-df_timelines_oppo['monthly_start']).dt.days
df_timelines_oppo['quarterly_duration'] = (df_timelines_oppo['quarterly_end']-df_timelines_oppo['quarterly_start']).dt.days

df_timelines_oppo.to_csv('../2024_Data/oppo/timeline_oppo.csv',index=False,encoding="utf-8")

df_timelines_oppo.head()


,phone,monthly_start,monthly_end,quarterly_start,quarterly_end,others_start,others_end,model_name,model_number,model_name_v2,monthly_duration,quarterly_duration
0,Find X2（CPH2023、CPH1921）,2022-10-11 18:51:30,2024-05-31 04:32:50,NaT,NaT,NaT,NaT,Find X2,CPH2023、CPH1921,Find X2,597.0,NaN
1,Find X2 Pro（CPH2025）,2022-10-11 18:51:30,2023-03-07 19:24:15,NaT,NaT,NaT,NaT,Find X2 Pro,CPH2025,Find X2 Pro,147.0,NaN
2,Find X3（PEDM00）,2022-10-11 18:51:30,2024-05-31 04:32:50,NaT,NaT,2021-01-21 17:16:59,2022-06-09 07:23:31,Find X3,PEDM00,Find X3,597.0,NaN
3,Find X3 Neo（CPH2207）,2022-10-11 18:51:30,2024-05-31 04:32:50,NaT,NaT,2021-01-21 17:16:59,2022-06-09 07:23:31,Find X3 Neo,CPH2207,Find X3 Neo,597.0,NaN
4,Find X3 Pro（PEEM00、CPH2173、CPH2305）,2022-10-11 18:51:30,2024-05-31 04:32:50,NaT,NaT,NaT,NaT,Find X3 Pro,PEEM00、CPH2173、CPH2305,Find X3 Pro,597.0,NaN


In [22]:
print('Unique Devices (all support lists): {}'.format(len(df_timelines_oppo['model_name_v2'].unique())))

Unique Devices (all support lists): 208


In [23]:
# Convert the string columns to datetime format
df_timelines_oppo['monthly_end'] = pd.to_datetime(df_timelines_oppo['monthly_end'])
df_timelines_oppo['quarterly_end'] = pd.to_datetime(df_timelines_oppo['quarterly_end'])
df_timelines_oppo['others_end'] = pd.to_datetime(df_timelines_oppo['others_end'])

# Get the maximum date from the three columns
max_date = df_timelines_oppo[['monthly_end', 'quarterly_end', 'others_end']].max().max()

# Filter the rows where the date in any of the three columns matches the max_date
matching_rows = df_timelines_oppo[
    (df_timelines_oppo['monthly_end'] == max_date) |
    (df_timelines_oppo['quarterly_end'] == max_date) |
    (df_timelines_oppo['others_end'] == max_date)
]

print('Unique Devices (last support list): {}'.format(len(matching_rows['model_name_v2'].unique())))

Unique Devices (last support list): 147


# 4. Google

In [25]:
import pandas as pd

df_timeline_google = pd.read_csv('../2024_Data/pixel/google_devices.csv')

df_timeline_google.head()

,id,device,last_android_version,last_update_date,guaranteed_android_version_updates,guaranteed_security_updates
0,sailfish,Pixel,10.0.0,12/1/19,18-Oct,19-Oct
1,walleye,Pixel 2,11.0.0,12/1/20,20-Oct,20-Oct
2,taimen,Pixel 2 XL,11.0.0,12/1/20,20-Oct,20-Oct
3,blueline,Pixel 3,12.0.0,6/1/22,21-Oct,21-Oct
4,crosshatch,Pixel 3 XL,12.0.0,6/1/22,21-Oct,21-Oct


In [26]:
df_timeline_pixel = df_timeline_google[df_timeline_google['device'].str.contains('Pixel')]
df_timeline_nexus = df_timeline_google[df_timeline_google['device'].str.contains('Nexus')]

print('Unique Pixel devices: {}'.format(len(df_timeline_pixel['device'].unique())))
print('Unique Nexus devices: {}'.format(len(df_timeline_nexus['device'].unique())))


Unique Pixel devices: 25
Unique Nexus devices: 9


In [27]:
#Devices support as of submission
df_timeline_pixel_supported = df_timeline_pixel[df_timeline_pixel['guaranteed_security_updates']>'2024-06-03']

print('Supported Pixel devices as of submission: {}'.format(len(df_timeline_pixel_supported)))

Supported Pixel devices as of submission: 20


# 5. Huawei

In [28]:
def get_support_timeline(device, dataset):

    df = dataset    
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df_device = df[df['device'] == device].sort_values(by='timestamp', ascending=True)
    
    try:
        monthly_start = df_device[df_device.period == 'monthly'].iloc[0]['timestamp']
        monthly_end = df_device[df_device.period == 'monthly'].iloc[-1]['timestamp']
    except IndexError:
        monthly_start = None
        monthly_end = None
        
    try: 
        quarterly_start = df_device[df_device.period == 'quarterly'].iloc[0]['timestamp']
        quarterly_end = df_device[df_device.period == 'quarterly'].iloc[-1]['timestamp']
    except IndexError:
        quarterly_start = None
        quarterly_end = None

    result = [monthly_start, monthly_end, quarterly_start, quarterly_end]
    
    return result

In [29]:
import pandas as pd

huawei_support_lists = pd.read_csv('../2024_Data/huawei/huawei_support_lists.csv')

# Filter the date to include on data by May 2024
huawei_support_lists['timestamp'] = pd.to_datetime(huawei_support_lists['timestamp'], format='%m/%d/%y')
huawei_support_lists = huawei_support_lists[huawei_support_lists['timestamp'] < '2024-06-01']

print('Unique devices:', len(set(huawei_support_lists['device'])))
counts_by_period = huawei_support_lists.groupby('period')['device'].nunique()
print(counts_by_period)
huawei_support_lists.head()

Unique devices: 257
period
monthly      110
quarterly    196
Name: device, dtype: int64


,timestamp,device,series,period
0,2019-06-04,P30 Pro,HUAWEI P series,monthly
1,2019-06-04,P30,HUAWEI P series,monthly
2,2019-06-04,"P30 lite,P20 Pro",HUAWEI P series,monthly
3,2019-06-04,P20,HUAWEI P series,monthly
4,2019-06-04,P20 lite,HUAWEI P series,monthly


In [30]:
df_timelines_huawei = pd.DataFrame()
for device in huawei_support_lists['device'].unique():
    support_timeline = get_support_timeline(device, huawei_support_lists)
    df_new = pd.DataFrame({"device": device, 
                           "monthly_start": support_timeline[0],
                           "monthly_end": support_timeline[1],
                           "quarterly_start": support_timeline[2],
                           "quarterly_end": support_timeline[3],
                          },
                          index=[0])
 
    df_new = df_new.dropna(axis=1, how='all')
    df_timelines_huawei = pd.concat([df_timelines_huawei, df_new], ignore_index=True)

df_timelines_huawei.to_csv('../2024_Data/huawei/timelines_huawei.csv', index=False)
df_timelines_huawei.head()


,device,monthly_start,monthly_end,quarterly_start,quarterly_end
0,P30 Pro,2019-06-04,2020-09-29,NaT,NaT
1,P30,2019-06-04,2020-09-29,NaT,NaT
2,"P30 lite,P20 Pro",2019-06-04,2019-06-04,NaT,NaT
3,P20,2019-06-04,2020-04-13,NaT,NaT
4,P20 lite,2019-06-04,2019-07-24,2019-10-12,2020-08-12


In [31]:
import pandas as pd

huawei_support_lists = huawei_support_lists.rename(columns={'timestamp': 'date'})
first_snapshot = huawei_support_lists['date'].min()
last_snapshot = huawei_support_lists['date'].max()
all_snapshots = len(huawei_support_lists['date'].unique())
avg_snapshot_interval = (last_snapshot - first_snapshot).days / all_snapshots
unique_devices_total = len(df_timelines_huawei.index)
unique_devices_last_list = len(huawei_support_lists[huawei_support_lists['date'] == last_snapshot])

print('First snapshot: {}'.format(first_snapshot))
print('Last snapshot: {}'.format(last_snapshot))
print('All snapshots: {}'.format(all_snapshots))
print('Avg snapshot interval (days): {}'.format(avg_snapshot_interval))
print('Unique Devices (Total): {}'.format(unique_devices_total))
print('Unique Devices (Last list): {}'.format(unique_devices_last_list))

df_timelines_huawei[['monthly_start','monthly_end', 'quarterly_start', 'quarterly_end']] = df_timelines_huawei[['monthly_start','monthly_end', 'quarterly_start', 'quarterly_end']].apply(pd.to_datetime)
df_timelines_huawei['monthly_duration'] = df_timelines_huawei['monthly_end']-df_timelines_huawei['monthly_start']
df_timelines_huawei['quarterly_duration'] = df_timelines_huawei['quarterly_end']-df_timelines_huawei['quarterly_start']

df_timelines_huawei.to_csv('../2024_Data/huawei/timelines_huawei.csv',index=False)

cond1 = ~(df_timelines_huawei['monthly_duration'].dt.days==0) #filter out if only one time appeared in the list
cond2 = ~df_timelines_huawei['monthly_duration'].isnull() #filter out as no monthly duration can be calculated
df_timelines_monthly = df_timelines_huawei[cond1&cond2].reset_index(drop=True)

cond3 = ~(df_timelines_huawei['quarterly_duration'].dt.days==0) #filter out if only one time appeared in the list
cond4 = ~df_timelines_huawei['quarterly_duration'].isnull() #filter out as no monthly duration can be calculated
df_timelines_quarterly = df_timelines_huawei[cond3&cond4].reset_index(drop=True)

print("Avg monthly support duration: {}".format(df_timelines_monthly['monthly_duration'].mean(skipna=True)))
print('Monthly completed periods for: {} devices'.format(len(df_timelines_monthly['device'].unique())))

print("Avg quarterly support duration: {}".format(df_timelines_quarterly['quarterly_duration'].mean(skipna=True)))
print('Quarterly completed periods for: {} devices'.format(len(df_timelines_quarterly['device'].unique())))

First snapshot: 2019-06-04 00:00:00
Last snapshot: 2024-05-28 00:00:00
All snapshots: 54
Avg snapshot interval (days): 33.7037037037037
Unique Devices (Total): 257
Unique Devices (Last list): 31
Avg monthly support duration: 310 days 13:11:12.527472528
Monthly completed periods for: 91 devices
Avg quarterly support duration: 326 days 06:46:46.779661016
Quarterly completed periods for: 177 devices


# 6. Vivo

In [32]:
import pandas as pd

def get_support_timeline(device, dataset):
    df = dataset.copy()
    
    df['date'] = pd.to_datetime(df['date'])
    df_device = df[df['device'] == device].sort_values(by='date', ascending=True)
    
    try:
        monthly_start = df_device[df_device.support_type == 'monthly'].iloc[0]['date']
        monthly_end = df_device[df_device.support_type == 'monthly'].iloc[-1]['date']
    except IndexError:
        monthly_start = None
        monthly_end = None
    try: 
        quarterly_start = df_device[df_device.support_type == 'quarterly'].iloc[0]['date']
        quarterly_end = df_device[df_device.support_type == 'quarterly'].iloc[-1]['date']
    except IndexError:
        quarterly_start = None
        quarterly_end = None
    try: 
        regular_start = df_device[df_device.support_type == 'regular'].iloc[0]['date']
        regular_end = df_device[df_device.support_type == 'regular'].iloc[-1]['date']
    except IndexError:
        regular_start = None
        regular_end = None

    result = [monthly_start, monthly_end, quarterly_start, quarterly_end, regular_start, regular_end]
    
    return result

In [33]:
import pandas as pd

vivo_support_lists = pd.read_csv('../2024_Data/vivo/vivo_support_lists.csv')

# Filter the date to include on data by May 2024
vivo_support_lists['date'] = pd.to_datetime(vivo_support_lists['date'])
vivo_support_lists = vivo_support_lists[vivo_support_lists['date'] < '2024-06-01']

print('Unique devices:', len(set(vivo_support_lists['device'])))
counts_by_period = vivo_support_lists.groupby('support_type')['device'].nunique()
print(counts_by_period)
vivo_support_lists.head()

Unique devices: 173
support_type
monthly      110
quarterly     52
regular       66
Name: device, dtype: int64


/var/folders/64/c68452391mv_849dpqtr2rch0000gn/T/ipykernel_60299/987395055.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  vivo_support_lists['date'] = pd.to_datetime(vivo_support_lists['date'])


,date,device,support_type
0,2022-10-05,iQOO 9,monthly
1,2022-10-05,iQOO 9 SE,monthly
2,2022-10-05,iQOO 9 Pro,monthly
3,2022-10-05,iQOO 9T,monthly
4,2022-10-05,iQOO 7,monthly


In [34]:
df_timelines_vivo = pd.DataFrame()
for device in vivo_support_lists['device'].unique():
    support_timeline = get_support_timeline(device, vivo_support_lists)
    df_new = pd.DataFrame({"device": device, 
                           "monthly_start": support_timeline[0],
                           "monthly_end": support_timeline[1],
                           "quarterly_start": support_timeline[2],
                           "quarterly_end": support_timeline[3],
                           "regular_start": support_timeline[4],
                           "regular_end": support_timeline[5]},
                          index=[0])
    df_new = df_new.dropna(axis=1, how='all')
    df_timelines_vivo = pd.concat([df_timelines_vivo, df_new], ignore_index=True)

df_timelines_vivo.head()

,device,monthly_start,monthly_end,regular_start,regular_end,quarterly_start,quarterly_end
0,iQOO 9,2022-10-05,2024-05-28,NaT,NaT,NaT,NaT
1,iQOO 9 SE,2022-10-05,2024-05-28,NaT,NaT,NaT,NaT
2,iQOO 9 Pro,2022-10-05,2024-05-28,NaT,NaT,NaT,NaT
3,iQOO 9T,2022-10-05,2024-05-28,NaT,NaT,NaT,NaT
4,iQOO 7,2022-10-05,2024-05-28,NaT,NaT,NaT,NaT


In [35]:
import pandas as pd

first_snapshot = vivo_support_lists['date'].min()
last_snapshot = vivo_support_lists['date'].max()
all_snapshots = len(vivo_support_lists['date'].unique())
avg_snapshot_interval = (last_snapshot - first_snapshot).days / all_snapshots
unique_devices_total = len(df_timelines_vivo.index)
unique_devices_last_list = len(vivo_support_lists[vivo_support_lists['date'] == last_snapshot])

print('First snapshot: {}'.format(first_snapshot))
print('Last snapshot: {}'.format(last_snapshot))
print('All snapshots: {}'.format(all_snapshots))
print('Avg snapshot interval (days): {}'.format(avg_snapshot_interval))
print('Unique Devices (Total): {}'.format(unique_devices_total))
print('Unique Devices (Last list): {}'.format(unique_devices_last_list))

df_timelines_vivo[['monthly_start','monthly_end', 'quarterly_start', 'quarterly_end', 'regular_start','regular_end']] = df_timelines_vivo[['monthly_start','monthly_end', 'quarterly_start', 'quarterly_end', 'regular_start','regular_end']].apply(pd.to_datetime)
df_timelines_vivo['monthly_duration'] = df_timelines_vivo['monthly_end']-df_timelines_vivo['monthly_start']
df_timelines_vivo['quarterly_duration'] = df_timelines_vivo['quarterly_end']-df_timelines_vivo['quarterly_start']
df_timelines_vivo['regular_duration'] = df_timelines_vivo['regular_end']-df_timelines_vivo['regular_start']

df_timelines_vivo.to_csv('../2024_Data/vivo/timelines_vivo.csv',index=False)

cond1 = ~(df_timelines_vivo['monthly_duration'].dt.days==0) #filter out if only one time appeared in the list
cond2 = ~df_timelines_vivo['monthly_duration'].isnull() #filter out as no monthly duration can be calculated
df_timelines_monthly = df_timelines_vivo[cond1&cond2].reset_index(drop=True)

cond3 = ~(df_timelines_vivo['quarterly_duration'].dt.days==0) #filter out if only one time appeared in the list
cond4 = ~df_timelines_vivo['quarterly_duration'].isnull() #filter out as no monthly duration can be calculated
df_timelines_quarterly = df_timelines_vivo[cond3&cond4].reset_index(drop=True)

cond5 = ~(df_timelines_vivo['regular_duration'].dt.days==0) #filter out if only one time appeared in the list
cond6 = ~df_timelines_vivo['regular_duration'].isnull() #filter out as no monthly duration can be calculated
df_timelines_regular = df_timelines_vivo[cond5&cond6].reset_index(drop=True)

print("Avg monthly support duration: {}".format(df_timelines_monthly['monthly_duration'].mean(skipna=True)))
print('Monthly completed periods for: {} devices'.format(len(df_timelines_monthly['device'].unique())))

print("Avg quarterly support duration: {}".format(df_timelines_quarterly['quarterly_duration'].mean(skipna=True)))
print('Quarterly completed periods for: {} devices'.format(len(df_timelines_quarterly['device'].unique())))

print("Avg regular support duration: {}".format(df_timelines_regular['regular_duration'].mean(skipna=True)))
print('Regular completed periods for: {} devices'.format(len(df_timelines_regular['device'].unique())))

First snapshot: 2022-10-05 00:00:00
Last snapshot: 2024-05-28 00:00:00
All snapshots: 8
Avg snapshot interval (days): 75.125
Unique Devices (Total): 173
Unique Devices (Last list): 175
Avg monthly support duration: 405 days 11:27:16.363636364
Monthly completed periods for: 88 devices
Avg quarterly support duration: 374 days 21:30:00
Quarterly completed periods for: 48 devices
Avg regular support duration: 443 days 06:22:02.448979592
Regular completed periods for: 49 devices


# 7. Motorola

In [36]:
import pandas as pd

def get_support_timeline(device, dataset):
    df = dataset.copy()
    
    df['date'] = pd.to_datetime(df['date'])
    df_device = df[df['device'] == device].sort_values(by='date', ascending=True)
    
    try:
        monthly_start = df_device[df_device.security_type == 'monthly'].iloc[0]['date']
        monthly_end = df_device[df_device.security_type == 'monthly'].iloc[-1]['date']
    except IndexError:
        monthly_start = None
        monthly_end = None
    
    try: 
        regular_start = df_device[df_device.security_type == 'regular'].iloc[0]['date']
        regular_end = df_device[df_device.security_type == 'regular'].iloc[-1]['date']
    except IndexError:
        regular_start = None
        regular_end = None

    result = [monthly_start, monthly_end, regular_start, regular_end]
    
    return result


In [37]:
import pandas as pd

motorola_support_lists = pd.read_csv('../2024_Data/motorola/motorola_support_lists.csv')

# Filter the date to include on data by May 2024
motorola_support_lists['date'] = pd.to_datetime(motorola_support_lists['date'].astype(str), format='%Y%m%d')
motorola_support_lists = motorola_support_lists[motorola_support_lists['date'] < '2024-06-01']

print('Unique devices:', len(set(motorola_support_lists['device'])))
counts_by_period = motorola_support_lists.groupby('security_type')['device'].nunique()
print(counts_by_period)
motorola_support_lists.head()

Unique devices: 17
security_type
monthly     2
regular    15
Name: device, dtype: int64


,device_family,device,country,date,security_type,launch_date,support_end_date,launch_OS,upgraded_to_OS,next_OS,note1,note2,link
0,android one family,moto g pro,Australia,2023-05-10,monthly,"june, 2020","june, 2023",NaN,NaN,NaN,NaN,channels and regions may have different launch...,https://web.archive.org/web/20230510/https://e...
1,moto g family,1 device,Australia,2023-05-10,regular,"april, 2023","april, 2026",android 13,NaN,android 14,this device will receive at least bimonthly sm...,channels and regions may have different launch...,https://web.archive.org/web/20230510/https://e...
2,moto e family,moto e40,Australia,2023-05-10,regular,"september, 2021","september, 2023",NaN,NaN,NaN,NaN,channels and regions may have different launch...,https://web.archive.org/web/20230510/https://e...
3,motorola edge family,1 device,Australia,2023-05-10,regular,"may, 2023","may, 2027",android 13,NaN,android 14,this device will receive at least bimonthly sm...,NaN,https://web.archive.org/web/20230510/https://e...
4,motorola one family,motorola one 5G UW ACE,Australia,2023-05-10,regular,"july, 2021","july, 2023",NaN,NaN,NaN,NaN,channels and regions may have different launch...,https://web.archive.org/web/20230510/https://e...


In [38]:
df_timelines_motorola = pd.DataFrame()
for device in motorola_support_lists['device'].unique():
    
    support_timeline = get_support_timeline(device, motorola_support_lists)
    
    df_new = pd.DataFrame({"device": device, 
                           "monthly_start": support_timeline[0],
                           "monthly_end": support_timeline[1],
                           "regular_start": support_timeline[2],
                           "regular_end": support_timeline[3]},
                          index=[0])
    df_new = df_new.dropna(axis=1, how='all')
    df_timelines_motorola = pd.concat([df_timelines_motorola, df_new], ignore_index=True)

df_timelines_motorola.head()

,device,monthly_start,monthly_end,regular_start,regular_end
0,moto g pro,2023-04-03,2023-07-03,NaT,NaT
1,1 device,NaT,NaT,2023-05-07,2024-04-15
2,moto e40,NaT,NaT,2023-04-03,2024-02-04
3,motorola one 5G UW ACE,NaT,NaT,2023-04-03,2023-07-03
4,motorola razr 2022,NaT,NaT,2023-04-03,2023-06-05


In [39]:
import pandas as pd

first_snapshot = motorola_support_lists['date'].min()
last_snapshot = motorola_support_lists['date'].max()
all_snapshots = len(motorola_support_lists['date'].unique())
avg_snapshot_interval = (last_snapshot - first_snapshot).days / all_snapshots
unique_devices_total = len(df_timelines_motorola.index)
unique_devices_last_list = len(motorola_support_lists[motorola_support_lists['date'] == last_snapshot])

print('First snapshot: {}'.format(first_snapshot))
print('Last snapshot: {}'.format(last_snapshot))
print('All snapshots: {}'.format(all_snapshots))
print('Avg snapshot interval (days): {}'.format(avg_snapshot_interval))
print('Unique Devices (Total): {}'.format(unique_devices_total))
print('Unique Devices (Last list): {}'.format(unique_devices_last_list))


First snapshot: 2023-04-03 00:00:00
Last snapshot: 2024-05-28 00:00:00
All snapshots: 59
Avg snapshot interval (days): 7.135593220338983
Unique Devices (Total): 17
Unique Devices (Last list): 5


In [40]:
df_timelines_motorola[['monthly_start','monthly_end','regular_start','regular_end']] = df_timelines_motorola[['monthly_start','monthly_end','regular_start','regular_end']].apply(pd.to_datetime)
df_timelines_motorola['monthly_duration'] = df_timelines_motorola['monthly_end']-df_timelines_motorola['monthly_start']
df_timelines_motorola['regular_duration'] = df_timelines_motorola['regular_end']-df_timelines_motorola['regular_start']

df_timelines_motorola.to_csv('../2024_Data/motorola/timelines_motorola.csv',index=False)

cond1 = ~(df_timelines_motorola['monthly_duration'].dt.days==0) #filter out if only one time appeared in the list
cond2 = ~df_timelines_motorola['monthly_duration'].isnull() #filter out as no monthly duration can be calculated
df_timelines_monthly = df_timelines_motorola[cond1&cond2].reset_index(drop=True)

cond3 = ~(df_timelines_motorola['regular_duration'].dt.days==0) #filter out if only one time appeared in the list
cond4 = ~df_timelines_motorola['regular_duration'].isnull() #filter out as no monthly duration can be calculated
df_timelines_regular = df_timelines_motorola[cond3&cond4].reset_index(drop=True)

print("Avg monthly support duration: {}".format(df_timelines_monthly['monthly_duration'].mean(skipna=True)))
print('Monthly completed periods for: {} devices'.format(len(df_timelines_monthly['device'].unique())))

print("Avg regular support duration: {}".format(df_timelines_regular['regular_duration'].mean(skipna=True)))
print('Regular completed periods for: {} devices'.format(len(df_timelines_regular['device'].unique())))

Avg monthly support duration: 256 days 00:00:00
Monthly completed periods for: 2 devices
Avg regular support duration: 131 days 03:25:42.857142858
Regular completed periods for: 14 devices


# 8. LG

In [45]:
import pandas as pd

lg_support_lists = pd.read_csv('../2024_Data/lg/lg_support_lists.csv')

# Filter the date to include on data by May 2024
lg_support_lists['date'] = pd.to_datetime(lg_support_lists['date'])
lg_support_lists = lg_support_lists[lg_support_lists['date'] < '2024-06-01']
lg_support_lists = lg_support_lists.rename(columns={'model':'device'})

print('Unique devices:', len(set(lg_support_lists['device'])))
# counts_by_period = lg_support_lists.groupby('support_type')['device'].nunique()
print(counts_by_period)
lg_support_lists.head()

Unique devices: 73
security_type
monthly     2
regular    15
Name: device, dtype: int64


/var/folders/64/c68452391mv_849dpqtr2rch0000gn/T/ipykernel_60299/1750586731.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  lg_support_lists['date'] = pd.to_datetime(lg_support_lists['date'])


,series,device,date
0,G series,G5,2021-12-13
1,G series,G6,2021-12-13
2,G series,G7,2021-12-13
3,G series,G8,2021-12-13
4,V series,V10,2021-12-13


In [46]:
import pandas as pd

lg_support_lists = pd.read_csv('../2024_Data/lg/lg_support_lists.csv')
print('Unique models:', len(set(lg_support_lists['model'])))
print('Unique dates:', len(set(lg_support_lists['date'])))

lg_support_lists['date'] = pd.to_datetime(lg_support_lists['date'])
lg_support_lists = lg_support_lists.sort_values(by='date')

earliest = lg_support_lists['date'].iloc[0]
latest = lg_support_lists['date'].iloc[-1]
print('Earliest Date:', earliest)
print('Latest Date:', latest)


Unique models: 73
Unique dates: 15
Earliest Date: 2021-12-13 00:00:00
Latest Date: 2024-03-02 00:00:00


/var/folders/64/c68452391mv_849dpqtr2rch0000gn/T/ipykernel_60299/2262078472.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  lg_support_lists['date'] = pd.to_datetime(lg_support_lists['date'])
